# 1. Auditoría de Datos: Tabla `customers`

Se realizó un perfilado inicial del archivo `olist_customers_dataset.csv` utilizando la librería Pandas para definir la arquitectura de la tabla en PostgreSQL.

## Hallazgos Técnicos:
* **Volumen de Datos:** 99,441 registros
* **Integridad de Columnas:** Se identificaron **5 columnas** con un 100% de valores no nulos (`non-null`)

### Análisis de Cardinalidad:
* **`customer_id`:** Posee 99,441 valores únicos, confirmando su aptitud como **Primary Key**.
* **`customer_unique_id`:** Posee 96,096 valores únicos, lo que indica la existencia de clientes recurrentes (re-compras).

In [7]:
import pandas as pd

df_customers = pd.read_csv('../data/olist_customers_dataset.csv')

data_quality = pd.DataFrame({
    'Tipo': df_customers.dtypes,
    'No Nulos': df_customers.count(),
    'Nulos': df_customers.isnull().sum(),
    'Valores Unicos': df_customers.nunique(),
    'Duplicados': len(df_customers) - df_customers.nunique()
})

data_quality

,Tipo,No Nulos,Nulos,Valores Unicos,Duplicados
customer_id,object,99441,0,99441,0
customer_unique_id,object,99441,0,96096,3345
customer_zip_code_prefix,int64,99441,0,14994,84447
customer_city,object,99441,0,4119,95322
customer_state,object,99441,0,27,99414


## 1.2. Definición de Tipos de Datos y Almacenamiento

Basado en el análisis de longitudes y naturaleza de los datos, se establecen las siguientes reglas de negocio para SQL:

* **Identificadores (`customer_id`, `unique_id`):** Longitud fija de 32 caracteres. Se usará `VARCHAR(50)` por flexibilidad.
* **`customer_city`:** Rango de 3 a 35 letras. Se usará `VARCHAR(100)`.
* **`customer_state`:** Longitud fija de 2 caracteres. Decisión técnica: `CHAR(2)`.
* **`customer_zip_code_prefix`:** Detectado como `int64`. Se usará `INTEGER`.

In [ ]:
##############################################################################################################

# SELECCIONAR LAS COLUMNAS OBJECT PARA DECIDIR EL TIPO DE DATO SERAN (CHAR, VARCHAR, NVARCHAR)

##############################################################################################################

df_analisys = df_customers.select_dtypes(include=['object'])\
                        .apply(lambda x: x.str.len())\
                        .agg(['min', 'max']).T\
                        .reset_index()

df_analisys.columns = ['Columna', 'Longitud Min', 'Longitud Max']

df_analisys

,Columna,Longitud Min,Longitud Max
0,customer_id,32,32
1,customer_unique_id,32,32
2,customer_city,3,32
3,customer_state,2,2
